# **Meta-Llama-3.1-8B-Instruct 모델 GPTQ 양자화하기**

# 1. 라이브러리 다운로드

In [ ]:
!pip install -q -U transformers peft accelerate optimum
!pip install datasets==2.15.0
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/
!pip install huggingface_hub
!pip install langchain
!pip install transformers[torch] -U

# 2. 지원되는 데이터셋을 전달하여 모델 양자화하기
- 'wikitest2' 데이터셋 사용
- 추후 재정정보 데이터셋 전처리 후 사용고려
- 4비트 정밀도로 양자화 (지원 : 2,4,6,8)

> **전처리 시 고려사항**
1. PyPDF2, pdfplumbler, pdfminer 같은 라이브러리로 pdf에서 텍스트 추출
2. 불필요한 공백, 페이지 번호, 제목 등 필요하지 않은 정보 제거
3. 순서가 맞지 않는 문장 재구성, 정제
4. 데이터셋 형태로 저장(JSON, CSV, TXT 등)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

# 모델 ID
model_id = "sh2orc/Llama-3.1-Korean-8B-Instruct" # 베이스 Llama 3 + 한국어 파인튜닝 모델

# GPTQ 양자화 설정
quantization_config = GPTQConfig(
    bits=4,
    group_size=128,       # 일반적으로 사용되는 그룹 크기
    dataset="wikitext2",  # 양자화에 사용할 기본 데이터셋, 추후 제공받은 데이터셋을 전처리하여 넣어주는 것도 고려하기
    desc_act=False
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델 로드 및 양자화
quant_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",         # 자동으로 가능한 장치에 할당
    torch_dtype=torch.float16  # 메모리 절약을 위해 float16 사용
)

선형 레이어의 속성을 확인하여 모델이 올바르게 정량화되었는지 확인할 수 있으며, 여기에는 torch.int32 dtype에 있어야 하는 qweight 및 qzeros 속성이 포함되어 있어야 합니다.

In [ ]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

이제 잘 동작하는지 확인하기 위해 양자화된 모델에 대해 추론을 수행해 보겠습니다. (트랜스포머와 동일한 API를 사용할 것)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

text = "안녕"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs)
print(tokenizer.decode(out[0], skip_special_tokens=True))

# 3. 양자화된 모델 허깅페이스에 업로드하기

In [ ]:
from huggingface_hub import notebook_login

# Hugging Face Hub 로그인 (최초 1회 실행 필요)
notebook_login()

In [ ]:
# 양자화된 모델을 Hugging Face Hub에 업로드
quant_model.push_to_hub("ormor/Llama-3.1-Korean-8B-Instruct-GPTQ-4bit")
tokenizer.push_to_hub("ormor/Llama-3.1-Korean-8B-Instruct-GPTQ-4bit")

# 4. 허깅페이스에 업로드한 모델 가져오기

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline

def setup_llm_pipeline():
    # 모델 ID (Hugging Face Hub에서 가져온 양자화된 모델 ID)
    model_id = "ormor/Llama-3.1-Korean-8B-Instruct-GPTQ-4bit"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        trust_remote_code=True
    )
    
    print(f"#### [ model ] ####\n{model}\n###################")

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=450,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

파이프라인 설정 및 테스트

In [ ]:
# 파이프라인 설정
hf_pipeline = setup_llm_pipeline()

# 텍스트 생성 예시
text = "안녕"
result = hf_pipeline(text)
print(result)